In [1]:
%matplotlib inline
from nd2reader import ND2Reader
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import h5py
from tifffile import imsave
matplotlib.rcParams['figure.figsize'] = [14, 10]

#### Part 1: ND2 to hdf5

I'd like the first step in the pipeline to convert the entire file to hdf5 so we can throw the nd2 away (maybe keep metadata)

I need both a slow local version with parallelization (low priority) and one that can be distributed to slurm


In [2]:
import dask
from dask.distributed import Client,progress
from dask_jobqueue import SLURMCluster
import time
import os

In [3]:
class hdf5_fov_extractor:
    def __init__(self,nd2filename,hdf5path):
        self.nd2filename = nd2filename
        self.hdf5path = hdf5path
        self.writedir(hdf5path)
    def writedir(self,directory,overwrite=False):
        if overwrite:
            if os.path.exists(directory):
                shutil.rmtree(directory)
            os.makedirs(directory)
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)
    def extract_fov(self,fovnum):
        nd2file = ND2Reader(self.nd2filename)
        metadata = nd2file.metadata
        with h5py.File(self.hdf5path + "/fov_" + str(fovnum) + ".hdf5", "w") as h5pyfile:
            for i,channel in enumerate(nd2file.metadata["channels"]):
                y_dim = metadata['height']
                x_dim = metadata['width']
                t_dim = len(nd2file.metadata['frames'])
                hdf5_dataset = h5pyfile.create_dataset("channel_" + str(channel),\
                                (x_dim,y_dim,t_dim),chunks=(x_dim,y_dim,1),dtype='uint16')
                for frame in nd2file.metadata['frames']:
                    print(frame)
                    nd2_image = nd2file.get_frame_2D(c=i, t=frame, v=fovnum)
                    hdf5_dataset[:,:,int(frame)] = nd2_image
        nd2file.close()
class tiff_fov_extractor:
    def __init__(self,nd2filename,tiffpath):
        self.nd2filename = nd2filename
        self.tiffpath = tiffpath
    def writedir(self,directory,overwrite=False):
        if overwrite:
            if os.path.exists(directory):
                shutil.rmtree(directory)
            os.makedirs(directory)
        else:
            if not os.path.exists(directory):
                os.makedirs(directory)
    def extract_fov(self,fovnum):
        nd2file = ND2Reader(self.nd2filename)
        metadata = nd2file.metadata
        for i,channel in enumerate(nd2file.metadata["channels"]):
            t_dim = len(nd2file.metadata['frames'])
            dirpath = self.tiffpath + "/fov_" + str(fovnum) + "/" + channel + "/"
            self.writedir(dirpath,overwrite=True)
            for frame in nd2file.metadata['frames']:
                filepath = dirpath + "t_" + str(frame) + ".tif"
                nd2_image = nd2file.get_frame_2D(c=i, t=frame, v=fovnum)
                imsave(filepath, nd2_image)
        nd2file.close()

In [4]:
hdf5_extractor = hdf5_fov_extractor("/n/scratch2/de64/for_sylvia/Bacillus_revival_12_7_2020.nd2","/n/scratch2/de64/full_pipeline_test/hdf5")

In [5]:
hdf5_extractor.extract_fov(1)

/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/nd2reader/common_raw_metadata.py:94: RuntimeWarning: Reported average frame interval (119915.3 ms) doesn't match the set interval (120000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)


0
1
2
3
4
5
6
7
8
9
10


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-5-236b4960292d>", line 1, in <module>
    hdf5_extractor.extract_fov(1)
  File "<ipython-input-3-dc8a00b1a177>", line 25, in extract_fov
    nd2_image = nd2file.get_frame_2D(c=i, t=frame, v=fovnum)
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/nd2reader/reader.py", line 77, in get_frame_2D
    return self._parser.get_image_by_attributes(t, v, c_name, z, y, x)
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/nd2reader/parser.py", line 98, in get_image_by_attributes
    height, width)
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/nd2reader/parser.py", line 256, in _get_raw_image_data
    data = read_chunk(self._fh, chunk)
  File "/home/de64/anaconda3/envs/mothe

KeyboardInterrupt: 

In [6]:
%load_ext line_profiler
%lprun -f hdf5_extractor.extract_fov hdf5_extractor.extract_fov(1)

/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/nd2reader/common_raw_metadata.py:94: RuntimeWarning: Reported average frame interval (119915.3 ms) doesn't match the set interval (120000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
*** KeyboardInterrupt exception caught in code being profiled.

Timer unit: 1e-06 s

Total time: 37.7137 s
File: <ipython-input-3-dc8a00b1a177>
Function: extract_fov at line 14

Line #      Hits         Time  Per Hit   % Time  Line Contents
    14                                               def extract_fov(self,fovnum):
    15         1     123539.0 123539.0      0.3          nd2file = ND2Reader(self.nd2filename)
    16         1          2.0      2.0      0.0          metadata = nd2file.metadata
    17         1      45680.0  45680.0      0.1          with h5py.File(self.hdf5path + "/fov_" + str(fovnum) + ".hdf5", "w") as h5pyfile:
    18         1          8.0      8.0      0.0              for i,channel in enumerate(nd2file.metadata["channels"]):
    19         1          2.0      2.0      0.0                  y_dim = metadata['height']
    20         1          1.0      1.0      0.0                  x_dim = metadata['width']
    21         1          2.0      2.0      0.0                  t_dim = len(nd2file.metadata['frames'])
    22        

In [4]:
class hdf5writer:
    def __init__(self,nd2filename,outputpath,n_workers=6,local=True,queue="short",\
                 walltime='01:30:00',cores=1,processes=1,memory='6GB'):
        self.nd2filename = nd2filename
        self.outputpath = outputpath
        self.local = local
        self.n_workers = n_workers
        self.walltime = walltime
        self.queue = queue
        self.processes = processes
        self.memory = memory
        self.cores = cores
    def writedir(self,directory):
        if not os.path.exists(directory):
            os.makedirs(directory)
    def startdask(self):
        if self.local:
            self.daskclient = Client()
            self.daskclient.cluster.scale(self.n_workers)
        else:
            # note the specifed walltime, don't use too much or too little, 01:30:00 is a good baseline, 
            # you just need enough time to finish 'gathering' to props_all before the jobs die
            # you can always spin up more jobs later
            # you will launch many jobs, so you don't need multiple processes, a lot of ram or multiple threads
            self.daskcluster = SLURMCluster(queue=self.queue,walltime=self.walltime,\
                                   processes=self.processes,memory=self.memory,
                                  cores=self.cores)
            self.workers = self.daskcluster.start_workers(self.n_workers)
            self.daskclient = Client(self.daskcluster)
    def printprogress(self):
        complete = len([item for item in self.futures if item.status=="finished"])
        print(str(complete) + "/" + str(len(self.futures)))
    def startwritehdf5(self):
        self.writedir(self.outputpath)
        extractor = hdf5_fov_extractor(self.nd2filename,self.outputpath)
        nd2file = ND2Reader(self.nd2filename)
        self.futures = self.daskclient.map(extractor.extract_fov,nd2file.metadata['fields_of_view'])
        nd2file.close()
    def startwritetiff(self):
        self.writedir(self.outputpath)
        extractor = tiff_fov_extractor(self.nd2filename,self.outputpath)
        nd2file = ND2Reader(self.nd2filename)
        self.futures = self.daskclient.map(extractor.extract_fov,nd2file.metadata['fields_of_view'])
        nd2file.close()

In [5]:
writer1 = hdf5writer("/n/scratch2/de64/for_sylvia/Bacillus_revival_12_7_2020.nd2",\
                     "/n/scratch2/de64/for_sylvia/tiff_out",\
                     walltime='04:00:00',local=False,n_workers=20,memory='500MB')
writer1.startdask()
writer1.daskcluster.start_workers()

In [7]:
writer1.daskclient

Client Scheduler: tcp://10.120.16.168:34015 Dashboard: http://10.120.16.168:8787/status,Cluster Workers: 5 Cores: 5 Memory: 2.50 GB


In [13]:
writer1.futures

[<Future: status: pending, key: extract_fov-0ee35abd91e2628e0de8852b32389495>,
 <Future: status: pending, key: extract_fov-dad4b36045180345e0ef6897efcd71ac>,
 <Future: status: pending, key: extract_fov-56a55fcd112b94b61d71818f0e7f0d3a>,
 <Future: status: pending, key: extract_fov-7603f02351f23cc2a25fa74589f12122>,
 <Future: status: pending, key: extract_fov-3896a5e72a6d5988a1523a1627702bd5>,
 <Future: status: pending, key: extract_fov-64c57478fba0c139cd3d2341e4ba9754>,
 <Future: status: pending, key: extract_fov-3bef4be4a30d101bc01a87158cd05f6e>,
 <Future: status: pending, key: extract_fov-882bda5ed6ed8645bc07b95d43209bce>,
 <Future: status: pending, key: extract_fov-f86349b2ffe7b2d51a54267e581fcfee>,
 <Future: status: pending, key: extract_fov-0e40419c31c627f74017d64148f566ff>,
 <Future: status: pending, key: extract_fov-edbbd42b72630fa500f0417d718e6b6b>,
 <Future: status: pending, key: extract_fov-5ace4c27fada10e722ba6ecf53c2ae47>,
 <Future: status: pending, key: extract_fov-100c816e

Exception ignored in: <bound method GraphPlot.__del__ of <distributed.bokeh.scheduler.GraphPlot object at 0x7f947877cbe0>>
Traceback (most recent call last):
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/distributed/bokeh/scheduler.py", line 741, in __del__
    self.scheduler.remove_plugin(self.layout)
  File "/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/distributed/scheduler.py", line 2172, in remove_plugin
    self.plugins.remove(plugin)
ValueError: list.remove(x): x not in list


In [8]:
writer1.startwritetiff()

/home/de64/anaconda3/envs/mothermachine/lib/python3.6/site-packages/nd2reader/common_raw_metadata.py:94: RuntimeWarning: Reported average frame interval (119915.3 ms) doesn't match the set interval (120000.0 ms). Using the average now.
  warnings.warn(message % (avg_interval, interval), RuntimeWarning)


In [15]:
writer1.printprogress()

6/40


In [29]:
props = writer1.daskclient.gather(writer1.futures) #this will hang until all futures are done

In [22]:
writer1.daskcluster.stop_workers(writer1.workers)  #this is still not working
writer1.daskcluster.stop_all_jobs() #this seems to work

In [79]:
with h5py.File('mytestfile.hdf5', 'r') as df:
    for fov in df:
        for frame in df[fov]:
            for color in df[fov+"/"+frame]:
                print(df[fov+"/"+frame + "/" + color][:])

[[556 566 597 ... 660 676 662]
 [485 618 464 ... 680 601 710]
 [615 562 521 ... 715 713 703]
 ...
 [789 789 764 ... 745 745 797]
 [846 805 688 ... 676 778 818]
 [827 830 918 ... 778 794 797]]
